
# ciência_de_dados_insights.ipynb

Notebook preparado para rodar localmente (ou no Google Colab). Objetivos:
- Calcular RFM e gerar clusters de clientes (K-Means)
- Prever vendas mensais (Holt-Winters)
- Exportar resultados para uso no Power BI (`data/clientes_clusters.csv`, `data/previsao_vendas.csv`)

**Observações**:
- Os arquivos CSV originais devem estar em `/mnt/data/` (já enviados por você):  
  `vendas_2023_2025.csv`, `clientes.csv`, `itens_venda.csv`, `produtos.csv`
- No Colab, faça upload dos CSVs para o ambiente ou monte o Google Drive.
- Se usar Prophet, instale `prophet` — aqui usamos `statsmodels` (Holt-Winters) para evitar dependências extras.


In [ ]:

# --- Setup e imports ---
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# Modelagem / clustering
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Forecasting
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Anomaly detection (opcional)
from sklearn.ensemble import IsolationForest

# Paths (ajuste se necessário)
PATH_VENDAS = '/mnt/data/vendas_2023_2025.csv'
PATH_CLIENTES = '/mnt/data/clientes.csv'
PATH_ITENS = '/mnt/data/itens_venda.csv'
PATH_PRODUTOS = '/mnt/data/produtos.csv'

print('Bibliotecas importadas.')



## 1. Carregar dados
Certifique-se de que os CSVs estejam nos caminhos acima. As colunas esperadas (resumo):
- `vendas_2023_2025.csv`: venda_id, cliente_id, data_venda, total_venda, ...
- `clientes.csv`: cliente_id, nome, email, cidade, estado, data_cadastro, ...
- `itens_venda.csv`: item_id, venda_id, produto_id, quantidade, preco_unitario, total_item
- `produtos.csv`: produto_id, nome, categoria, preco, custo


In [ ]:

# Carregar CSVs (com parsing de datas onde aplicável)
vendas = pd.read_csv(PATH_VENDAS, parse_dates=['data_venda'], dayfirst=False, infer_datetime_format=True)
clientes = pd.read_csv(PATH_CLIENTES)
itens = pd.read_csv(PATH_ITENS)
produtos = pd.read_csv(PATH_PRODUTOS)

print('Linhas carregadas:')
print('vendas:', vendas.shape)
print('clientes:', clientes.shape)
print('itens:', itens.shape)
print('produtos:', produtos.shape)

# Merge para um dataframe analítico (cada item de venda em uma linha)
df = (vendas.merge(clientes, on='cliente_id', how='left', suffixes=('','_cliente'))
        .merge(itens, on='venda_id', how='left', suffixes=('','_item'))
        .merge(produtos, on='produto_id', how='left', suffixes=('','_produto')))

print('Dataframe combinado df:', df.shape)
df.head(3)



## 2. RFM (Recency, Frequency, Monetary) e K-Means

- **Recency**: dias desde a última compra (utilizamos snapshot = data máxima das vendas + 1 dia)
- **Frequency**: número de compras (vendas) por cliente
- **Monetary**: soma do valor gasto por cliente

Depois normalizamos e aplicamos K-Means para segmentação.


In [ ]:

# RFM
snapshot = vendas['data_venda'].max() + pd.Timedelta(days=1)
print('Snapshot (data referência):', snapshot.date())

# Frequency: contar vendas distintas por cliente
frequency = vendas.groupby('cliente_id')['venda_id'].nunique().rename('Frequency')
# Recency: dias desde última compra
recency = vendas.groupby('cliente_id')['data_venda'].max().apply(lambda x: (snapshot - x).days).rename('Recency')
# Monetary: soma total_venda por cliente
monetary = vendas.groupby('cliente_id')['total_venda'].sum().rename('Monetary')

rfm = pd.concat([recency, frequency, monetary], axis=1).reset_index()
rfm.describe().round(2)


In [ ]:

# Preparar para K-Means: transformação log (para Monetary) e padronização
rfm_clean = rfm.copy()
rfm_clean['Monetary_log'] = np.log1p(rfm_clean['Monetary'])
features = ['Recency','Frequency','Monetary_log']

scaler = StandardScaler()
X = scaler.fit_transform(rfm_clean[features])

# Escolher k com elbow (rápido)
sse = []
K = range(2,7)
for k in K:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X)
    sse.append(km.inertia_)

import matplotlib.pyplot as plt
plt.figure(figsize=(6,3))
plt.plot(K, sse, '-o')
plt.xlabel('k')
plt.ylabel('SSE (inertia)')
plt.title('Elbow method (k selection)')
plt.tight_layout()

# Definimos k=4 (ajuste conforme o elbow)
k = 4
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
rfm_clean['Cluster'] = kmeans.fit_predict(X)

# Mapear cluster para rótulos qualitativos (exemplo)
cluster_order = rfm_clean.groupby('Cluster')['Monetary'].median().sort_values(ascending=False).index.tolist()
label_map = {cluster_order[0]: 'VIP', cluster_order[1]: 'Loyal', cluster_order[2]: 'Occasional', cluster_order[3]: 'At Risk'}
rfm_clean['Cluster_Label'] = rfm_clean['Cluster'].map(label_map)

rfm_clean[['cliente_id','Recency','Frequency','Monetary','Cluster','Cluster_Label']].head(8)



### Exportar clusters para Power BI
O arquivo `data/clientes_clusters.csv` será consumido no Power BI (crie uma página "Customer Segments").


In [ ]:

# Garantir pasta data/
os.makedirs('/mnt/data', exist_ok=True)
rfm_out = rfm_clean[['cliente_id','Recency','Frequency','Monetary','Cluster','Cluster_Label']].copy()
# Se quiser, juntar colunas de nome do cliente
rfm_out = rfm_out.merge(clientes[['cliente_id','nome','cidade','estado']], on='cliente_id', how='left')
rfm_out.to_csv('/mnt/data/clientes_clusters.csv', index=False)
print('Exportado: /mnt/data/clientes_clusters.csv (linhas = {})'.format(len(rfm_out)))



## 3. Previsão de Vendas Mensais (Holt-Winters)

- Agrupamos vendas por mês (soma de `total_venda`) e aplicamos ExponentialSmoothing (Holt-Winters) com sazonalidade anual (12 meses).
- Exportamos previsão para uso no Power BI.


In [ ]:

# Agrupar vendas por mês
mensal = vendas.set_index('data_venda').resample('M')['total_venda'].sum().rename('y')
mensal = mensal.to_frame().reset_index()
mensal.head(6)


In [ ]:

# Ajustar Holt-Winters com sazonalidade aditiva (12 meses)
ts = mensal.copy()
ts = ts.set_index('data_venda').asfreq('M')
ts['y'] = ts['y'].fillna(0)

model = ExponentialSmoothing(ts['y'], trend='add', seasonal='add', seasonal_periods=12, initialization_method='estimated')
fit = model.fit(optimized=True)

# Previsão para próximos 6 meses
future_periods = 6
forecast_index = pd.date_range(start=ts.index[-1] + pd.offsets.MonthBegin(1), periods=future_periods, freq='M')
forecast_vals = fit.forecast(future_periods)

forecast_df = pd.DataFrame({'ds': forecast_index, 'yhat': forecast_vals.values}).reset_index(drop=True)

# Combine histórico + forecast for plotting
hist_df = ts.reset_index().rename(columns={'data_venda':'ds','y':'y'})
plot_df = pd.concat([hist_df.tail(36), forecast_df], ignore_index=True)

import matplotlib.pyplot as plt
plt.figure(figsize=(8,3.5))
plt.plot(hist_df['ds'], hist_df['y'], label='Histórico')
plt.plot(forecast_df['ds'], forecast_df['yhat'], label='Previsão', linestyle='--')
plt.legend()
plt.title('Vendas Mensais - Histórico e Previsão (6 meses)')
plt.tight_layout()

# Export forecast CSV
forecast_out = forecast_df[['ds','yhat']].copy()
forecast_out.to_csv('/mnt/data/previsao_vendas.csv', index=False)
print('Exportado: /mnt/data/previsao_vendas.csv (linhas = {})'.format(len(forecast_out)))



## 4. (Opcional) Detecção de Anomalias nas Transações
Usamos IsolationForest em features simples (total_item, preco_unitario) para sinalizar vendas possivelmente atípicas.


In [ ]:

# Preparar amostra de transações (itens)
itens_sample = df[['venda_id','item_id','quantidade','preco_unitario','total_item','data_venda']].dropna()

# Features para isolation forest: total_item e quantidade (poderá ajustar)
X_anom = itens_sample[['quantidade','total_item']].copy()
iso = IsolationForest(contamination=0.002, random_state=42)
itens_sample['anomaly_score'] = iso.fit_predict(X_anom)
# -1 -> anomalia, 1 -> normal
anomalies = itens_sample[itens_sample['anomaly_score'] == -1].sort_values('total_item', ascending=False).head(20)
anomalies[['venda_id','item_id','quantidade','total_item']].head(10)



## Próximos passos / Integração com Power BI

- Importe `/mnt/data/clientes_clusters.csv` para criar páginas de **Segmentação de Clientes**.
- Importe `/mnt/data/previsao_vendas.csv` e combine com a série histórica para visualizar previsões.
- Adicione slicers (Ano, Categoria, Estado) e cards com KPIs (ticket médio, vendas totais, clientes ativos).
- Salve o notebook em `notebooks/` e faça commit no repositório GitHub.
